In [5]:
import requests      # Для запросов по API
import json          # Для обработки полученных результатов
import time          # Для задержки между запросами
import os            # Для работы с файлами
import pandas as pd  # Для формирования датафрейма с результатами\
import pprint   

Для получения всех стран со всеми их внутренними зонами

In [2]:
def getAreas():
    req = requests.get('https://api.hh.ru/areas')    
    data = req.content.decode()
    req.close()
    jsObj = json.loads(data)
    areas = []
    for k in jsObj:
        for i in range(len(k['areas'])):
            if len(k['areas'][i]['areas']) != 0:                      # Если у зоны есть внутренние зоны
                for j in range(len(k['areas'][i]['areas'])):
                    areas.append([k['id'], 
                                  k['name'], 
                                  k['areas'][i]['areas'][j]['id'],
                                  k['areas'][i]['areas'][j]['name']])
            else:                                                                # Если у зоны нет внутренних зон
                areas.append([k['id'], 
                              k['name'], 
                              k['areas'][i]['id'], 
                              k['areas'][i]['name']])
    return areas

areas = getAreas()

Следующим шагом стоит найти ID работодателей.

Для этого нужно получить количество работодателей на данный момент и учесть тот факт, что не все порядковые номера существуют и внутренние ограничения API HH на постраничный поиск, глубина которого равна всего 2000 значений.

In [8]:
def getEmployers():
    req = requests.get('https://api.hh.ru/employers')
    data = req.content.decode()
    req.close()
    count_of_employers = json.loads(data)['found']
    employers = []
    i = 0
    j = count_of_employers
    while i < j:
            req = requests.get('https://api.hh.ru/employers/'+str(i+1))
            data = req.content.decode()
            req.close()
            jsObj = json.loads(data)
            try:
                employers.append([jsObj['id'], jsObj['name']])
                i += 1
                print([jsObj['id'], jsObj['name']])
            except:
                i += 1
                j += 1
            if i%200 == 0:
                time.sleep(0.2)
    return employers
    
employers = getEmployers()

['1', 'NCI']
['6', 'ИДС Боржоми']
['13', 'WiseAdvice']
['14', 'People You Need']
['15', 'Наука-Связь']
['19', 'КВАРТА ВК']
['25', 'Deutsche Telekom IT Solutions']
['26', 'Хлебный дом']
['35', 'МОСС']
['36', 'АДЛ']
['37', 'ВИКИНГ, банк']
['40', 'Amphora Group']
['41', 'ЮНЭКТ']
['42', 'АВТОАУДИОЦЕНТР']
['44', 'Акрон']
['47', 'Каре']
['48', 'Аналитик-Центр']
['49', 'Микротест']
['51', 'Всероссийский Банк Развития Регионов']
['54', 'СП БИЗНЕС КАР']
['56', 'Отличные Окна']
['59', 'РОЛЬФ, группа компаний']
['61', 'Союзподшипник']
['63', 'Стальпрокат']
['64', 'RRC']
['65', 'Московский аэропорт Домодедово']
['67', 'AMS']
['68', 'Фолиум']
['69', 'Порт.ру']
['71', 'ВОБИС КОМПЬЮТЕР']
['76', 'Ericsson']
['79', 'NCR']
['80', 'Альфа-Банк']
['82', 'МКД']
['83', 'Западная Техника']
['87', 'BrandLab']
['89', 'БАНК УРАЛСИБ']
['91', 'Ферреро Руссия']
['92', 'Новое Содружество']
['93', 'Интермедика']
['98', 'ОПТИМА']
['99', 'РусьРегионБанк, Санкт-Петербургский филиал']
['100', 'НПФ Радио-Сервис']
['108', 

KeyboardInterrupt: 

Возьмем для примера 2ГИС с ID 64174 и найдем все вакансии по работодателю в разрезе каждой зоны России (ID 113). В функцию getPage в качестве входных параметров сделаем только номер страницы для постраничного поиска и зону, где будем смотреть вакансии.

In [9]:
def getPage(page, area):
    params = {
        'employer_id': 3529,  # ID 2ГИС
        'area': area,         # Поиск в зоне
        'page': page,         # Номер страницы
        'per_page': 100       # Кол-во вакансий на 1 странице
    }   
    req = requests.get('https://api.hh.ru/vacancies', params)
    data = req.content.decode() 
    req.close()
    return data

Часть кода, где функция getPage и используется:

In [11]:
for id, area in enumerate(areas):
    for page in range(0, 20):
        jsObj = json.loads(getPage(page, area[2]))
        if not os.path.exists('./areas/'):
            os.makedirs('./areas/')
        nextFileName = './areas/{}.json'.format(str(area[2])+'_'+str(area[3])+'_'+str(page))
        f = open(nextFileName, mode='w', encoding='utf8')
        f.write(json.dumps(jsObj, ensure_ascii=False))
        f.close()
        if (jsObj['pages'] - page) <= 1:  
            print('[{0}/{1}] Область: {3} ({2}) - {5} ({4}) Вакансий: {6}'.format(id+1, 
                                                                         len(areas), 
                                                                         area[0], 
                                                                         area[1], 
                                                                         area[2], 
                                                                         area[3], 
                                                                         jsObj['found']))
            break
    time.sleep(0.2)

[1/5508] Область: Россия (113) - Виловатово (4228) Вакансий: 0
[2/5508] Область: Россия (113) - Волжск (1621) Вакансий: 1
[3/5508] Область: Россия (113) - Звенигово (1622) Вакансий: 0
[4/5508] Область: Россия (113) - Знаменский (4229) Вакансий: 0
[5/5508] Область: Россия (113) - Йошкар-Ола (61) Вакансий: 9
[6/5508] Область: Россия (113) - Кельмаксола (4230) Вакансий: 0
[7/5508] Область: Россия (113) - Килемары (4231) Вакансий: 0
[8/5508] Область: Россия (113) - Козьмодемьянск (1623) Вакансий: 1
[9/5508] Область: Россия (113) - Красногорский (Республика Марий Эл) (4232) Вакансий: 0
[10/5508] Область: Россия (113) - Краснооктябрьский (4233) Вакансий: 0
[11/5508] Область: Россия (113) - Куженер (4234) Вакансий: 0
[12/5508] Область: Россия (113) - Мари-Турек (4235) Вакансий: 0
[13/5508] Область: Россия (113) - Медведево (4236) Вакансий: 0
[14/5508] Область: Россия (113) - Морки (3667) Вакансий: 0
[15/5508] Область: Россия (113) - Новый Торъял (4237) Вакансий: 0
[16/5508] Область: Россия (1

KeyError: 'pages'

Сохраняем промежуточные результаты в формате json для каждой зоны отдельно, в том числе и для зон, где не найдено ни одной вакансии. Теперь сгруппируем их в один файл:

In [ ]:
dt = []
for fl in os.listdir('./areas/'):
    f = open('./areas/{}'.format(fl), encoding='utf8')
    jsonText = f.read()
    f.close()
    jsonObj = json.loads(jsonText)
    if jsonObj['found'] != 0:
        for js in jsonObj['items']:
            if js['salary'] != None:
                salary_from = js['salary']['from']
                salaty_to = js['salary']['to']
            else:
                salary_from = None
                salaty_to = None
            if js['address'] != None:
                address_raw = js['address']['raw']
            else:
                address_raw = None
            dt.append([
                js['id'],
                js['premium'],
                js['name'],
                js['department']['name'],
                js['has_test'],
                js['response_letter_required'],
                js['area']['id'],
                js['area']['name'],
                salary_from, 
                salaty_to,
                js['type']['name'],
                address_raw,
                js['response_url'],
                js['sort_point_distance'],
                js['published_at'],
                js['created_at'],
                js['archived'],
                js['apply_alternate_url'],
                js['insider_interview'],
                js['url'],
                js['alternate_url'],
                js['relations'],
                js['employer']['id'],
                js['employer']['name'],
                js['snippet']['requirement'],
                js['snippet']['responsibility'],
                js['contacts'],
                js['schedule']['name'],
                js['working_days'],
                js['working_time_intervals'],
                js['working_time_modes'],
                js['accept_temporary']
                ])

Полученный промежуточный результат сохраняем в DataFrame и сохраняем как файл Excel.

In [ ]:
df = pd.DataFrame(dt, columns = [
                                'id',
                                'premium',
                                'name',
                                'department_name',
                                'has_test',
                                'response_letter_required',
                                'area_id',
                                'area_name',
                                'salary_from', 
                                'salaty_to',
                                'type_name',
                                'address_raw',
                                'response_url',
                                'sort_point_distance',
                                'published_at',
                                'created_at',
                                'archived',
                                'apply_alternate_url',
                                'insider_interview',
                                'url',
                                'alternate_url',
                                'relations',
                                'employer_id',
                                'employer_name',
                                'snippet_requirement',
                                'snippet_responsibility',
                                'contacts',
                                'schedule_name',
                                'working_days',
                                'working_time_intervals',
                                'working_time_modes',
                                'accept_temporary'
                                ])   
df.to_excel('result_2gis.xlsx')